In [18]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

from detection import showOpticalFlow
def gradients(I1, I2):
    Ix = cv2.Sobel(I1, cv2.CV_64F, 1, 0, ksize=5)  # Gradient en x
    Iy = cv2.Sobel(I1, cv2.CV_64F, 0, 1, ksize=5)  # Gradient en y
    It = I2 - I1  # Gradient temporel
    return Ix, Iy, It

def compute_weights(window_size):
    #w = np.array([[1/12,1/6,1/12],[1/6,1/6,1/6],[1/12,1/6,1/12]])
    w = 1/(window_size**2)*np.ones((window_size,window_size))
    # w = np.array([
    # [0.00296902, 0.01330621, 0.02193823, 0.01330621, 0.00296902],
    # [0.01330621, 0.0596343,  0.09832033, 0.0596343,  0.01330621],
    # [0.02193823, 0.09832033, 0.16210282, 0.09832033, 0.02193823],
    # [0.01330621, 0.0596343,  0.09832033, 0.0596343,  0.01330621],
    # [0.00296902, 0.01330621, 0.02193823, 0.01330621, 0.00296902]
#])    
    return w.flatten()

def lucas_kanade(image1, image2, window_size=5):
    # Calculer les gradients
    Ix, Iy, It = gradients(image1, image2)
    
    # Taille du voisinage
    w = window_size // 2
    
    # Préparer les vecteurs de flux
    u = np.zeros(image1.shape)
    v = np.zeros(image1.shape)
    
    # Calculer les poids
    weights = compute_weights(window_size)
    W = np.diag(weights)
    
    for i in range(w, image1.shape[0] - w):
        for j in range(w, image1.shape[1] - w):
            # Extraire le voisinage
            Ix_window = Ix[i-w:i+w+1, j-w:j+w+1].flatten()
            Iy_window = Iy[i-w:i+w+1, j-w:j+w+1].flatten()
            It_window = It[i-w:i+w+1, j-w:j+w+1].flatten()
            
            A = np.vstack((Ix_window, Iy_window)).T
            b = -It_window
            
            # Calculer la matrice M avec les poids
            M = A.T @ W @ W @ A
            
            # Calculer le vecteur b avec les poids
            B = A.T @ W @ W @ b
            l = 1
            # Résoudre le système d'équations
            U, S, V = np.linalg.svd(M, full_matrices=True)
            D = np.diag(S / (S**2 + l))
            X = V @ D @ np.transpose(U) @ B
            u[i, j] = X[0]
            v[i, j] = X[1]
            # if np.linalg.det(M) != 0:
            #     nu = np.linalg.pinv(M) @ nu
            #     u[i, j] = nu[0]
            #     v[i, j] = nu[1]
            # else : 
            #     print("d")
    
    return u, v


# Assurez-vous que les chemins vers les images sont corrects
path_to_image1 = 'MiniCooper/i0001.png'  # Remplacez par le chemin correct vers votre image 1
path_to_image2 = 'MiniCooper/i0002.png'  # Remplacez par le chemin correct vers votre image 2

# Chargez les images
I1 = cv2.imread(path_to_image1, 0)/255.0
I2 = cv2.imread(path_to_image2, 0)/255.0
u, v = lucas_kanade(I1, I2)



In [20]:
showOpticalFlow(I1, u, v, scaling_factor=100, sampling_rate=5, thresh=0.05)